In [5]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time, math
import scipy.stats as stats

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


basePath = '../MSRAction3D/skeleton/'

## Carrega instancia do arquivo
def loadData(action, subject, instance):
	ins = np.loadtxt(basePath + 'a%02i_s%02i_e%02i_skeleton.txt'%(action, subject, instance))
	ins = ins.reshape((ins.shape[0]//20, 20, 4))
	return ins

## Cria dicionario com toda a base de dados
def loadBase():
	base = {}
	for action in range(1, 21):
		base[action] = {}
		for subject in range(1, 11):
			base[action][subject] = {}
			for instance in [1, 2, 3]:
				try:
					data = loadData(action, subject, instance)
				except IOError:
					continue
				base[action][subject][instance] = data

	return base


base = loadBase()

In [6]:
## Padroniza o numero de frames
## Cria tabela com instancias por frames por componentes(PCA)
def formatTable(action, coords, n_frames):
	alpha = 0.5
	table = []
	pca = PCA(n_components=2)
	for subject in base[action]:
		for instance in base[action][subject]:

			frames = [frame[:, coords].T.flat[:] for frame in base[action][subject][instance]]
			
			##padronizar nro de frames n_frames
			a, b = math.ceil((len(frames)-n_frames)/2), (len(frames)-n_frames)//2
			#for i in range(1, table.size-1):
			#	stat, p = stats.kruskal(table.loc[i, :], table.loc[i-1, :])
			
			## Pega os n_frames centrais 
			if b>0:
				frames = frames[a:-b]
			elif a>0:
				frames = frames[a:]
			
			
			## Reduz valores para 2 componentes com PCA
			x = StandardScaler().fit_transform(frames)
			principalComponents = pca.fit_transform(x)
			
			## Cada linha da tabela corresponde a uma instancia da base  
			table.append(list(principalComponents.T.flat))
			
	table = pandas.DataFrame(table)
	return table

n_frames = 31
## Gera tabela com ação 1, coords x, y e z e nframes
table = formatTable(1, [0, 1, 2], n_frames)

In [7]:

## Renomeia as colunas da tabela
cols = []
for i in range(n_frames):
    cols.append('x%d'%(i))
    cols.append('y%d'%(i))
table.columns = cols
print(table)

           x0         y0         x1         y1        x2        y2        x3  \
0   -4.771645  -5.061021  -4.792742  -4.986284 -5.122662 -5.041734 -4.719271   
1    9.822265   9.763644   9.228249   8.926672  8.924597  8.015431  6.764265   
2   -7.038811  -7.290066  -7.224445  -7.285314 -7.191408 -7.028478 -6.669217   
3    7.161721   7.637743  13.745104  14.044312  5.746370  6.236129  5.840420   
4   -0.678376  -0.693696  -1.149605  -3.738279 -3.391407 -2.431126 -0.190680   
5   10.727095  10.240965  10.723371   7.691128  5.428054  6.547908  6.936972   
6   -3.328387  -4.158526  -4.226002  -3.946083 -4.327232 -4.413919 -4.166302   
7   -4.029174  -4.711385  -5.001736  -5.355099 -4.736177 -6.261323 -6.361906   
8   -4.963865  -4.403228  -4.286948  -4.421411 -4.254487 -4.671081 -4.333612   
9   -5.297970  -4.807739  -5.338615  -4.910432 -5.956264 -5.764812 -4.636765   
10  -5.151047  -4.755950  -4.148330  -4.352026 -4.319890 -4.573616 -4.232263   
11  -5.231652  -5.117054  -5.079622  -4.